In [1]:
from datetime import datetime, timedelta
import random, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Variable Initialization ##

In [2]:
# Lane Name List
laneName = ['c1', 'r1', 'c2', 'r2', 'c3', 'r3', 'c4', 'r4']
laneNameAll = ['c1', 'r1', 'l1','c2', 'r2', 'l2', 'c3', 'r3', 'l3', 'c4', 'r4', 'l4']

# Stores number of Car in each lane
carNumber = {}
for lane in laneNameAll:
    carNumber[lane] = 0

# Stores probablity by which car will spawn in a lane
laneProbab = {'c1' : 0.8,
             'r1' : 0.8,
             'l1' : 0.8,
             'c2' : 0.8,
             'r2' : 0.8,
             'l2' : 0.8,
             'c3' : 0.8,
             'r3' : 0.8,
             'l3' : 0.8,
             'c4' : 0.8,
             'r4' : 0.8,
             'l4' : 0.8}

# Lane Pattern
# Stores list of lanes which can be turned green simultaneously with corresponding lane
lanePattern = {'c1' : ['r1', 'r2', 'c3'],
               'r1' : ['c1', 'c4', 'r3'],
               'c2' : ['r2', 'r3', 'c4'],
               'r2' : ['c2', 'c1', 'r4'],
               'c3' : ['r3', 'r4', 'c1'],
               'r3' : ['c3', 'c2', 'r1'],
               'c4' : ['r4', 'r1', 'c2'],
               'r4' : ['c4', 'c3', 'r2']}

# Lane Flags
# Stores weather a pattern if switched green in a loop or not
laneFlags = {}
for lane1 in laneName:
    for lane2 in lanePattern[lane1]:
        laneFlags[lane1 + lane2] = False

# Number of Car in Node
# Stores Nodal weight of each lane
nodeCar = {}
for lane in laneName:
    nodeCar[lane] = 0
    
# light Status
# Variable to tell if Traffic is on or not for a lane
lightStatus = {}
for lane in laneName:
    lightStatus[lane] = False
    
# Variable to Store Time Took to execute
deltaTime = []

## Helper Functions ##

In [3]:
# Return True or False with passed probability
def decision(probability):
    return random.random() < probability

# Turn all traffic light to red[0]
def setLightRed():
    for a in laneName:            
        lightStatus[a] = False

# Add traffic 
def addTraffic():
    for lane in laneNameAll:
        if decision(laneProbab[lane]):
            carNumber[lane] += 1

# Add traffic, 'n' number of times
def addTrafficNumber(n):
    for a in range(n):
        addTraffic()
        
# Check if Number of cars is zero / Empty Traffic junc.
def check():
    s = 0
    for a in laneName:
        s = s + carNumber[a]
    return s != 0

## Algorithm and it's Performance calcuation ##

In [4]:
# Loop for 1000 times to collect 1000 datapoints
for abc in range(1000):
    addTrafficNumber(10)
    
    # Get Algo Starting Time
    timeStart = timedelta(seconds=0)
    timeStop = timeStart

    # Loop till number of car in junc. is zero
    while check():

        # Calculate Number of Cars Assosiated with One Node
        for a in laneName:
            s = 0  
            for b in lanePattern[a]:
                s = s + carNumber[b]
            nodeCar[a] = (carNumber[a] * 3) + s
            
        # Get List of Node with Sorted Number of Car Assosiated with it
        maxNode = sorted(nodeCar, key = nodeCar.get, reverse = True)

        # Variable for Looping Through List maxNode
        i = 0
        while i < 8:
            # Get Pattern List to Switch to
            # List is in Sorted Order
            maxPattern = sorted(lanePattern[maxNode[i]], key = carNumber.get, reverse = True)
        
            # Variable for Looping Through List maxPattern
            j = 0
            while j < 3:
                # Check Flag
                if laneFlags[maxNode[i] + maxPattern[j]] == True:
                    j = j + 1
            
                else:
                    # Set Flag to 1
                    laneFlags[maxNode[i] + maxPattern[j]] = True
                    laneFlags[maxPattern[j] + maxNode[i]] = True 
                        
                    # Check For Condition When Both Lane have Zero Cars
                    if carNumber[maxNode[i]] != 0 or carNumber[maxPattern[j]] != 0:
                        # Turn All Traffic Light to Red
                        # Turn Pattern Traffic Light to Green
                        if(lightStatus[maxNode[i]] == True and lightStatus[maxPattern[j]] == True):
                            if(carNumber[maxNode[i]] > 0):
                                carNumber[maxNode[i]] = carNumber[maxNode[i]] - 1
                            if(carNumber[maxPattern[j]] > 0):
                                carNumber[maxPattern[j]] = carNumber[maxPattern[j]] - 1

                        elif(lightStatus[maxNode[i]] == True and lightStatus[maxPattern[j]] == False):
                            setLightRed()
                            lightStatus[maxNode[i]] = True
                            lightStatus[maxPattern[j]] = True
                            if(carNumber[maxNode[i]] > 0):
                                carNumber[maxNode[i]] = carNumber[maxNode[i]] - 1
                            if(carNumber[maxPattern[j]] > 0):
                                carNumber[maxPattern[j]] = carNumber[maxPattern[j]] - 1

                        elif(lightStatus[maxNode[i]] == False and lightStatus[maxPattern[j]] == True):
                            setLightRed()
                            lightStatus[maxNode[i]] = True
                            lightStatus[maxPattern[j]] = True
                            if(carNumber[maxNode[i]] > 0):
                                carNumber[maxNode[i]] = carNumber[maxNode[i]] - 1
                            if(carNumber[maxPattern[j]] > 0):
                                carNumber[maxPattern[j]] = carNumber[maxPattern[j]] - 1

                        else:
                            setLightRed()
                            lightStatus[maxNode[i]] = True
                            lightStatus[maxPattern[j]] = True
                            if(carNumber[maxNode[i]] > 0):
                                carNumber[maxNode[i]] = carNumber[maxNode[i]] - 1
                            if(carNumber[maxPattern[j]] > 0):
                                carNumber[maxPattern[j]] = carNumber[maxPattern[j]] - 1
                        
                        # Add time taken by first car to move
                        timeStop += timedelta(seconds=1)
                            
                            
                        # Remove more cars based on number of cars based in the lanes
                        # if both lane have more than max number of cars that can pass
                        # green until max number of car passes
                        if (min([carNumber[maxNode[i]], carNumber[maxPattern[j]]]) > 7):
                            for a in range(0, 7):
                                timeStop += timedelta(seconds=1)
                                if(carNumber[maxNode[i]] > 0):
                                    carNumber[maxNode[i]] = carNumber[maxNode[i]] - 1
                                if(carNumber[maxPattern[j]] > 0):
                                    carNumber[maxPattern[j]] = carNumber[maxPattern[j]] - 1
                        
                        # If one/both lane have more than zero cars, but less than max number
                        else:
                            # Green signal untill one of the lane exhausts
                            if min([carNumber[maxNode[i]], carNumber[maxPattern[j]]]) != 0:
                                for a in range(0, min([carNumber[maxNode[i]], carNumber[maxPattern[j]]])):
                                    timeStop += timedelta(seconds=1)
                                    if(carNumber[maxNode[i]] > 0):
                                        carNumber[maxNode[i]] = carNumber[maxNode[i]] - 1
                                    if(carNumber[maxPattern[j]] > 0):
                                        carNumber[maxPattern[j]] = carNumber[maxPattern[j]] - 1
                                        
                            # If one of the lane has zero car
                            else:
                                # non-zero lane has more than max number of cars
                                # Green signal untill max number of cars passes
                                if max([carNumber[maxNode[i]], carNumber[maxPattern[j]]]) > 7:
                                    for a in range(0, 7):
                                        timeStop += timedelta(seconds=1)
                                        if(carNumber[maxNode[i]] > 0):
                                            carNumber[maxNode[i]] = carNumber[maxNode[i]] - 1
                                        if(carNumber[maxPattern[j]] > 0):
                                            carNumber[maxPattern[j]] = carNumber[maxPattern[j]] - 1
                                            
                                # non-zero lane has less than max number of cars
                                else:
                                    # Green signal untill cars in other lane exhausts
                                    for a in range(0, max([carNumber[maxNode[i]], carNumber[maxPattern[j]]])):
                                        timeStop += timedelta(seconds=1)
                                        if(carNumber[maxNode[i]] > 0):
                                            carNumber[maxNode[i]] = carNumber[maxNode[i]] - 1
                                        if(carNumber[maxPattern[j]] > 0):
                                            carNumber[maxPattern[j]] = carNumber[maxPattern[j]] - 1
                        
                        # Yellow light delay
                        timeStop += timedelta(seconds=4)
                                            
                    # Set J so that outter loop knows Traffic light is switched on
                    j = 10
                        
            if j == 10:
                # Set I so that outter loop knows Traffic light is switched on
                i = 10
                
            else:
                i = i + 1
                
        # Reset Flag if All Set to True
        if i == 8:
            for c in laneFlags.keys():
                laneFlags[c] = False
                
    delta = timeStop - timeStart
    deltaTime.append(delta)
    
timeList = []
for a in deltaTime:
    timeList.append(a.seconds)
    
deltaSec = pd.DataFrame(timeList, columns=['Time [Performance]'])

In [5]:
# For max car in each lane = 10
# Low Traffic
deltaSec.mean()

Time [Performance]    65.79
dtype: float64

In [44]:
# For max car in each lane = 40
# High Traffic
deltaSec.mean()

Time [Performance]    212.577
dtype: float64